In [78]:
!pip install -q requests

In [79]:
import nltk
nltk.download('stopwords')    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kashif/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [80]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
# Number of documents to scrape
N = 10 


categories = ['sports', 'education']



documents = []

# Scrape N/2 documents for each category
for cat in tqdm(categories):

    for i in range(N//2):
        
        # Construct Wikipedia url
        url = f'https://wikipedia.org/wiki/Special:Random'
        
        # Get page content
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        # Get document text
        text = soup.get_text()
        
        # Preprocess
        text = text.lower() # lowercase
        text = re.sub(r'\[[0-9]*\]',' ',text) 
        text = re.sub(r'\s+',' ',text)
        
        # Remove punctuation
        text = re.sub(r'[^\w\s]','',text)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word not in stop_words])

        documents.append(text)

print(len(documents), 'documents scraped and preprocessed')

100%|██████████| 2/2 [00:27<00:00, 13.89s/it]

10 documents scraped and preprocessed


In [81]:
# from langchain.retrievers import WikipediaRetriever
# retriever = WikipediaRetriever()

# query = ["Politics", "Health"]

# def remove_special_characters(text):
#     # Use regex to remove non-alphanumeric characters
#     text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
#     text = text.lower() # lowercase
#     # text = re.sub(r'\[[0-9]*\]',' ',text) 
#     text = re.sub(r'\s+',' ',text)
    
#     # Remove punctuation
#     text = re.sub(r'[^\w\s]','',text)
    
#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     text = ' '.join([word for word in text.split() if word not in stop_words])

#     return text

# documents = []

# for q in tqdm(query):
#     doc = retriever.get_relevant_documents(query=q)[0]
#     no_spec = remove_special_characters(doc.dict()["page_content"])
#     documents.append(no_spec)

In [93]:
documents

['eidertreene depression wikipedia jump content main menu main menu move sidebar hide navigation main pagecontentscurrent eventsrandom articleabout wikipediacontact usdonate contribute helplearn editcommunity portalrecent changesupload file languages language links top page search search create account log personal tools create account log pages logged editors learn contributionstalk contents move sidebar hide top 1sources 2external links toggle table contents eidertreene depression 2 languages danskdeutsch edit links articletalk english readeditview history tools tools move sidebar hide actions readeditview history general links hererelated changesupload filespecial pagespermanent linkpage informationcite pageget shortened urldownload qr codewikidata item printexport download pdfprintable version wikipedia free encyclopedia landscape west schleswigholstein north germany delver koog nature reserve bend river eider eidertreene depression german eidertreeneniederung danish ejdertrenesænk

In [83]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize

# Extract unigram features 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)



In [94]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize

# Extract unigram features 
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(documents)

# Unigram count matrix
unigram_count = X.toarray()

# Bigram probability matrix
bigram = CountVectorizer(ngram_range=(1,2), analyzer='word').fit(documents)
X_bigram = bigram.transform(documents)
sum_cols = X_bigram.sum(axis=0)
bigram_prob = normalize(X_bigram, norm='l1', axis=1) 

# TF-IDF matrix
tfidf = TfidfTransformer().fit_transform(X)

print('Unigram count matrix shape:', unigram_count.shape)
print('Bigram probability matrix shape:', bigram_prob.shape)
print('TF-IDF matrix shape:', tfidf.shape)

Unigram count matrix shape: (10, 3010)
Bigram probability matrix shape: (10, 8528)
TF-IDF matrix shape: (10, 3010)


In [95]:
import re
import nltk
from nltk.util import ngrams
from collections import Counter
import numpy as np

# Tokenize documents
tokenizer = nltk.RegexpTokenizer(r'\w+')
docs_tokens = [tokenizer.tokenize(doc) for doc in documents]

# Get vocabulary
vocab = set()
for doc_tokens in docs_tokens:
    vocab.update(doc_tokens)
vocab = list(vocab)

# Unigram count matrix
unigram_count = [[doc_tokens.count(word) for word in vocab] for doc_tokens in docs_tokens]

# Bigram probability matrix
bigrams = [list(ngrams(doc,2)) for doc in docs_tokens]
bigrams_freq = Counter([b for doc in bigrams for b in doc])
bigrams_prob = [[bigrams_freq[b]/sum(bigrams_freq.values()) for b in set(ngrams(doc,2))] for doc in docs_tokens]

# TF-IDF matrix 
doc_freq = {word: sum([1 for doc in docs_tokens if word in doc]) for word in vocab}
idf = {word: np.log(len(documents)/doc_freq[word]) for word in vocab}
tfidf = [[count * idf[word] for word, count in Counter(doc).items()] for doc in docs_tokens]

print(unigram_count[0][:5]) 
print(bigrams_prob[0][:5])
print(tfidf[0][:5])

[0, 1, 1, 0, 0]
[0.0001258970162407151, 0.001258970162407151, 0.0001258970162407151, 0.001258970162407151, 0.0001258970162407151]
[11.51292546497023, 13.815510557964275, 0.0, 0.0, 0.0]


In [96]:
documents

['eidertreene depression wikipedia jump content main menu main menu move sidebar hide navigation main pagecontentscurrent eventsrandom articleabout wikipediacontact usdonate contribute helplearn editcommunity portalrecent changesupload file languages language links top page search search create account log personal tools create account log pages logged editors learn contributionstalk contents move sidebar hide top 1sources 2external links toggle table contents eidertreene depression 2 languages danskdeutsch edit links articletalk english readeditview history tools tools move sidebar hide actions readeditview history general links hererelated changesupload filespecial pagespermanent linkpage informationcite pageget shortened urldownload qr codewikidata item printexport download pdfprintable version wikipedia free encyclopedia landscape west schleswigholstein north germany delver koog nature reserve bend river eider eidertreene depression german eidertreeneniederung danish ejdertrenesænk

In [97]:
y = [0,1,1,0,1,1,1,0,1,1]
# y = [0,1]

In [98]:
import numpy as np

class NaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.vocab = None
        self.classes = None
        self.prior = None
        self.likelihood = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.vocab = set()
        for doc in X:
            self.vocab.update(doc)
        self.vocab = list(self.vocab)

        n_instances = len(X)
        n_classes = len(self.classes)
        n_vocab = len(self.vocab)


        self.prior = np.zeros(n_classes)
        for c in self.classes:
            self.prior[c] = np.sum(y == c) / n_instances


        self.likelihood = np.zeros((n_classes, n_vocab))
        for c in self.classes:
            class_docs = [doc for doc, cls in zip(X, y) if cls == c]
            total_words = sum(len(doc) for doc in class_docs)
            word_counts = np.zeros(n_vocab)
            for doc in class_docs:
                for word in doc:
                    word_counts[self.vocab.index(word)] += 1
            self.likelihood[c] = (word_counts + self.alpha) / (total_words + n_vocab * self.alpha)

    def predict(self, X, use_unigram=True, use_bigram=True, use_tfidf=True):
        predictions = []
        for doc in X:
            scores = np.log(self.prior)
            unseen_class_score = np.log(1 / len(self.classes))  # Default score for unseen classes
            for word in doc:
                if use_unigram:
                    if word in self.vocab:
                        scores += np.log(self.likelihood[:, self.vocab.index(word)])
                    else:
                        scores += unseen_class_score
            if use_bigram:
                bigrams = list(ngrams(doc, 2))
                for bigram in bigrams:
                    if bigram in bigram_prob:
                        scores += np.log(bigram_prob[bigram])
                    else:
                        scores += unseen_class_score
            if use_tfidf:
                tfidf_val = np.array([tfidf[doc.index(word)] for word in doc if word in self.vocab])
                for i, score in enumerate(scores):
                    if len(tfidf_val) > 0:
                        scores[i] += np.sum(tfidf_val * np.log(self.likelihood[i, [self.vocab.index(word) for word in doc if word in self.vocab]]))
                    else:
                        scores[i] += unseen_class_score * sum(1 for word in doc if word not in self.vocab)
            predictions.append(self.classes[np.argmax(scores)])
        return predictions

In [89]:
# Prepare data
X = unigram_count 
y = [0,1,1,0,1,1,1,0,1,1]



nb = NaiveBayes()
nb.fit(X, y)


predictions = nb.predict(documents, use_unigram=True, use_bigram=True, use_tfidf=False)

In [99]:
predictions = nb.predict(documents, use_unigram=True, use_bigram=False, use_tfidf=False)
predictions

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [100]:
predictions = nb.predict(documents, use_unigram=False, use_bigram=True, use_tfidf=False)
predictions

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [101]:
predictions = nb.predict(documents, use_unigram=False, use_bigram=False, use_tfidf=True)
predictions

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]